## 深度学习自然语言处理第三次作业

##### 李明昕 SY2206124

### 1. LDA 算法



LDA（Latent Dirichlet Allocation，潜在狄利克雷分配）是一种常用的主题模型算法。主题模型是一种文本挖掘技术，可以帮助我们自动地发现一组文档中隐藏的主题，并识别出每个文档与这些主题之间的关系。

#### 1.1 原理


LDA算法假定文档的主题分布是一个多项分布（记其参数为 $\vec{\theta}$），而每个主题的单词分布也是一个多项分布（记其参数为 $\vec{\phi}$），并且这些多项分布的参数都服从狄利克雷分布（参数分别为 $\vec{\alpha}$ 和 $\vec{\beta}$）。

具体来说，在 LDA 算法中，生成文档的过程如下：

1. 按照先验概率 $p(d)$ 选择一篇文档 $d_m$；
2. 从狄利克雷分布 $\text{Dir}(\vec{\alpha})$ 中采样生成文档 $d_m$ 的主题分布的参数 $\vec{\theta}_m$；
3. 根据参数为 $\vec{\theta}_m$ 的多项分布 $\text{Mult}(\vec{\theta}_m)$ 为第 $n$ 个词采样生成一个主题 $z_{m,n}$；
4. 从狄利克雷分布 $\text{Dir}(\vec{\beta})$ 中采样生成主题 $z_{m,n}$ 的单词分布的参数 $\vec{\phi}_{z_{m,n}}$；
5. 根据参数为 $\vec{\phi}_{z_{m,n}}$ 的多项分布 $\text{Mult}(\vec{\phi}_{z_{m,n}})$ 为第 $n$ 个词采样生成一个单词 $w_{m,n}$。

![LDA_process](resources/LDA_process.png)

##### 1.1.1 联合概率分布

假设语料库中有 $M$ 个文档，每个文档的主题序列序列以及单词序列分别为 $Z=\{\vec{z}_1, \cdots, \vec{z}_M\}$ 和 $W=\{\vec{w}_1,\cdots,\vec{w}_M\}$。此外，$M$ 个文档的主题分布参数记为 $\Theta=\{\vec{\theta}_1, \cdots, \vec{\theta}_M\}$，$K$ 个主题的单词分布记为 $\Phi=\{\vec{\phi}_1,\cdots,\vec{\phi}_M\}$。并且，主题分布与单词分布的先验分布的参数分别为 $\vec{\alpha}$ 和 $\vec{\beta}$。

则 LDA 模型的联合概率分布可以表示为$^{[1]}$：
$$
p(W,Z,\Theta,\Phi|\vec{\alpha}, \vec{\beta}).
$$

其边缘概率分布可以表示为：
$$
\begin{aligned}
  p(W, Z | \vec{\alpha}, \vec{\beta})&= p(W | Z, \vec{\beta})p(Z|\vec{\alpha}) \\
  &= \prod_{k=1}^K\frac{\Delta(\vec{n}_k+\vec{\beta})}{\Delta(\vec{\beta})}\prod_{m=1}^M\frac{\Delta(\vec{n}_m+\vec{\alpha})}{\Delta(\vec{\alpha})},
\end{aligned}
$$
其中 $\Delta(\vec{x})$ 是迪利克雷分布的归一化因子，$\vec{n}_k=\{n_{k1},\cdots,n_{kV}\}$ 表示第 $k$ 个主题中单词的计数，$\vec{n}_m=\{n_{m1},\cdots,n_{mK}\}$。

后验概率可以表示为：

$$
\begin{aligned}
  p(Z | W, \vec{\alpha}, \vec{\beta}) &= \frac{p(W, Z | \vec{\alpha}, \vec{\beta})}{p(W | \vec{\alpha}, \vec{\beta})} \\
  &\propto \prod_{k=1}^K\frac{\Delta(\vec{n}_k+\vec{\beta})}{\Delta(\vec{\beta})}\prod_{m=1}^M\frac{\Delta(\vec{n}_m+\vec{\alpha})}{\Delta(\vec{\alpha})}.
\end{aligned}
$$

#### 1.2 吉布斯采样算法

##### 1.2.1 算法原理

通过对后验概率分布 $p(Z|W,\alpha,\beta)$ 进行吉布斯采样得到其样本集合。然后利用这个样本集合对参数 $\Theta$ 以及 $\Phi$ 进行估计。

$p(Z|W,\alpha,\beta)$ 的满条件概率分布可以写成
$$
\begin{aligned}
  p(z_i|Z_{-i}, W,\alpha,\beta) 
  &= \frac{1}{A_{z_i}}p(Z|W,\alpha,\beta) \\
  &\propto \frac{n_{kv}+\beta_v}{\sum_{v=1}^V(n_{kv}+\beta_v)}\cdot\frac{n_{mk}+\alpha_k}{\sum_{k=1}^K(n_{mk}+\alpha_k)},
\end{aligned}
$$
其中 $A_{z_i}$ 表示对变量 $z_i$ 的边缘化因子，$w_i$ 的是单词集合中的第 $v$ 个单词，$z_i$ 是主题集合中的第 k 个主题。$n_{kv}$ 表示第 $k$ 个词表的第 $v$ 个单词的计数（但去掉当前单词的计数），$n_{mk}$ 表示第 $m$ 个文档的第 $k$ 个主题的计数（但去掉当前主题的计数）。

通过满条件概率分布 $p(z_i|Z_{-i}, W,\alpha,\beta)$ 就可以进行吉布斯采样得到 $Z$ 的分配值，也可以得到 $\Theta$ 和 $\Phi$ 的估计值：

1. $\Theta=\{\theta_m\}$ 的估计：
   $\theta_m$ 的后验概率为：
   $$
   p(\theta_m|\vec{z}_m, \vec{\alpha})=\text{Dir}(\theta_m|\vec{n_m}+\vec{\alpha}),
   $$
   所以 $\Theta=\{\theta_m\}$ 的估计式为：
   $$
   \theta_{mk}=\frac{n_{mk}+\alpha_k}{\sum_{k=1}^K(n_{mk}+\alpha_k)}, m=1,2,\cdots,m;\ k=1,2,\cdots,K;
   $$
2. $\Phi=\{\phi_k\}$ 的估计：
   $\phi_k$ 的后验概率为：
   $$
    p(\phi_k|W,Z,\vec{\beta})=\text{Dir}(\phi_k|\vec{n_k}+\vec{\beta}),
   $$
   所以 $\Phi=\{\phi_k\}$ 的估计式为：
   $$
   \phi_{kv}=\frac{n_{kv}+\beta_v}{\sum_{v=1}^V(n_{kv}+\beta_v)}, k=1,2,\cdots,K;\ v=1,2,\cdots,V.
   $$

##### 1.2.2 算法过程

输入：文本的单词序列 $W=\{\vec{w}_1,\cdots,\vec{w}_M\}$，$\vec{w}_m=\{w_{m1},\cdots, w_{mN_m}\}$；
输出：文本的话题序列 $Z=\{\vec{z}_1,\cdots,\vec{z}_M\}$，$\vec{z}_m=\{z_{m1},\cdots, z_{mN_m}\}$ 的后验概率分布 $p(Z|W,\alpha,\beta)$ 的样本计数，模型的参数 $\Theta=\{\theta_m\}$，$\Phi=\{\phi_k\}$ 的估计值；
参数：超参数 $\alpha$, $\beta$，话题个数 $K$。
1. 设所有计数矩阵的元素 $n_{mk}, n_{kv}$ ，以及计数向量 $\vec{n}_m, \vec{n}_k$ 的元素都为 0；
2. 对所有文本 $\vec{w}_m,m=1,2,\cdots,M$
   对第 $m$ 个文本的所有单词 $w_{mn}$ ，$n=1,2,\cdots,N_m$：
   
   a. 采样话题 $z_{mn}=z_k\sim\text{Mult}(\frac{1}{K})$；
   
   b. 增加文本-话题计数 $n_{mk} = n_{mk} + 1$；
   
   c. 增加文本-话题和计数 $n_m=n_m+1$；
   
   d. 增加话题-单词计数 $n_{kv}=n_{kv}+1$；
   
   e. 增加话题-单词和计数 $n_k=n_k+1$；
3. 循环进行以下操作，直到进入燃烧器
   对所有文本 $\vec{w}_m$ ，对第 $m$ 个文本中的所有单词 $w_{mn}$：

   a. 当前单词 $w_{mn}$ 是第 $v$ 个单词，主题 $z_{mn}$ 是第 $k$ 个主题。减少计数 $n_{mk}=n_{mk}-1, n_m=n_m-1, n_{kv}=n_{kv}-1, n_k=n_k-1$；
   
   b. 按照满条件分布进行采样：
      $$
      p(z_i'|Z_{-i},W,\vec{\alpha},\vec{\beta})\propto\frac{n_{kv}+\beta_v}{\sum_{v=1}^V(n_{kv}+\beta_v)}\cdot\frac{n_{mk}+\alpha_k}{\sum_{k=1}^K(n_{mk}+\alpha_k)}
      $$
      得到新的第 $k'$ 个话题，分配给 $z_{mn}$
      
   c. 增加计数 $n_{mk'}=n_{mk'}+1, n_m=n_m+1, n_{k'v}=n_{k'v}+1, n_{k'}=n_{k'}+1$；
4. 利用得到的样本计数，计算模型参数
   $$
   \theta_{mk}=\frac{n_{mk}+\alpha_k}{\sum_{k=1}^K(n_{mk}+\alpha_k)};
   $$
   $$
   \phi_{kv}=\frac{n_{kv}+\beta_v}{\sum_{v=1}^V(n_{kv}+\beta_v)}.
   $$

### 2. 实现

In [2]:
import random
import jieba
import pdb
import re
import os
import numpy as np
from tqdm import tqdm
from functools import reduce
from typing import Tuple, List, Callable, Union
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

random.seed(0)

#### 2.1 预处理

In [3]:
class PreProcess:

    def __init__(self, data_dir: str, catalog_name: str, stopword_name: str, tokenize_mode='char'):
        self.data_dir = data_dir
        with open(os.path.join(self.data_dir, catalog_name), 'r', encoding='gb2312', errors='ignore') as fi:
            self.book2index = {book: i for i, book in enumerate(fi.read().strip().split(','))}
            self.index2book = {i: book for book, i in self.book2index.items()}
            print("Available books and their indices:")
            for book, index in self.book2index.items():
                print(f"{book}: {index}")
        self.stopword_path = os.path.join(data_dir, stopword_name)
        self.tokenize_mode = tokenize_mode
        self._read_books()
        self._tokenize()

    def _read_books(self):
        self.book2content = {}
        ad_p = re.compile(r"本书来自www.cr173.com免费txt小说下载站\n更多更新免费电子书请关注www.cr173.com")
        b_p = re.compile(r"\s")
        nc_p = re.compile(r"[^\u4e00-\u9fa5，…：、。！？；]")
        print('Read and parse books:')
        for book in self.book2index:
            print(f'{book}, ', end='')
            with open(os.path.join(self.data_dir, f'{book}.txt'), 'r', encoding='gb2312', errors='ignore') as fi:
                content = fi.read()
                content = ad_p.sub('', content)
                lines = [nc_p.sub('', b_p.sub('', line)) for line in content.strip().split('\n')]
                self.book2content[book] = lines
        print('Done reading and parsing.')


    def _tokenize(self):
        print(f"Tokenize in {self.tokenize_mode} mode:") 
        tokens = set()
        self.book2token_count = {}
        for book, content in self.book2content.items():
            tokenized_content = []
            token_count = 0
            for para in content:
                if self.tokenize_mode == 'char':
                    tokenized_content.append(list(para))
                else:
                    tokenized_content.append(list(jieba.cut(para)))
                token_count += len(tokenized_content[-1])
                tokens.update(tokenized_content[-1])
            self.book2content[book] = tokenized_content
            self.book2token_count[book] = token_count
            print(f"Token number of {book}: {token_count}")
        self.token2idx = {token: i for i, token in enumerate(tokens)}
        self.idx2token = {i: token for token, i in self.token2idx.items()}
        print(f"Vocabulary size: {len(tokens)}")
    
    def sample_corpus(self, para_num: int = 200, min_token_num: int = 500, max_token_num: int = 1000):
        book_seq = []
        weights = []
        for book, token_count in self.book2token_count.items():
            book_seq.append(book)
            weights.append(token_count)
        book_para4sample = {}
        for book in self.book2token_count:
            book_para4sample[book] = []
            for para_idx, para in enumerate(self.book2content[book]):
                book_para4sample[book].append(len(para))
        corpus = []
        while len(corpus) < para_num:
            book = random.choices(book_seq, weights=weights)[0]
            para_idx = random.choices(range(len(book_para4sample[book])), weights=book_para4sample[book])[0]
            para_tokens = self.book2content[book][para_idx]
            while len(para_tokens) < min_token_num and para_idx < len(book_para4sample[book]) - 1:
                para_idx += 1
                para_tokens.extend(self.book2content[book][para_idx])
            if len(para_tokens) >= min_token_num and len(para_tokens) <= max_token_num:
                corpus.append((para_tokens, book))
        idx_corpus = []
        for para, book in corpus:
            idx_corpus.append(([self.token2idx[token] for token in para], self.book2index[book]))
        return idx_corpus
            

#### 2.2 LDA 模型

In [4]:
class LDA:
    def __init__(self, alpha: np.ndarray, beta: np.ndarray):
        self.alpha = alpha
        self.beta = beta
        self.K = self.alpha.shape[0]
        self.V = self.beta.shape[0]

    def fit(self, corpus: List[Union[List, int]], max_iter: int = 100, min_topic_change: int = 100):
        # initialization
        W = [para for para, _ in corpus]
        self.M = len(W)
        self.Theta = np.zeros((self.M, self.K), dtype=int)
        self.n_m = np.zeros((self.M,), dtype=int)
        self.Phi = np.zeros((self.K, self.V), dtype=int)
        self.n_k = np.zeros((self.K,), dtype=int)
        Z = []
        for m, w_m in enumerate(W):
            Z.append([])
            for w_mn in w_m:
                z_mn = random.sample(range(self.K), k=1)[0]
                Z[-1].append(z_mn)
                self.Theta[m, z_mn] += 1
                self.n_m[m] += 1
                self.Phi[z_mn, w_mn] += 1
                self.n_k[z_mn] += 1
        
        # iteration
        with tqdm(range(max_iter)) as pbar:
            for _ in pbar:
                topic_change_count = 0
                for m, w_m in enumerate(W):
                    for n, w_mn in enumerate(w_m):
                        z_mn = Z[m][n]
                        self.Theta[m][z_mn] -= 1
                        self.n_m[m] -= 1
                        self.Phi[z_mn][w_mn] -= 1
                        self.n_k[z_mn] -= 1
                        weights = [(self.Phi[k, w_mn].item() + self.beta[w_mn].item()) / (self.n_k[k].item() + self.beta.sum()) * 
                                   (self.Theta[m, k].item() + self.alpha[k].item()) / (self.n_m[m].item() + self.alpha.sum())  
                                   for k in range(self.K)]
                        n_z_mn = random.choices(range(self.K), weights=weights, k=1)[0]
                        Z[m][n] = n_z_mn
                        self.Theta[m][n_z_mn] += 1
                        self.n_m[m] += 1
                        self.Phi[n_z_mn][w_mn] += 1
                        self.n_k[n_z_mn] += 1
                        if n_z_mn != z_mn:
                            topic_change_count += 1
                pbar.set_postfix({'topic_change_count': topic_change_count})
                if topic_change_count <= min_topic_change:
                    break
            
    def get_theta(self):
        if not hasattr(self, 'theta'):
            self.theta = np.empty(self.Theta.shape, dtype=np.float)
            for m in range(self.M):
                for k in range(self.K):
                    self.theta[m,k] = (self.Theta[m, k].item() + self.alpha[k].item()) / (self.n_m[m].item() + self.alpha.sum())
        return self.theta

    def get_phi(self):
        if not hasattr(self, 'phi'):
            self.phi = np.empty(self.Phi.shape, dtype=np.float)
            for k in range(self.K):
                for v in range(self.V):
                    self.phi[k,v] = (self.Phi[k, v].item() + self.beta[v].item()) / (self.n_k[k].item() + self.beta.sum())
        return self.phi

#### 2.3 SVM 分类

使用 SVM（Support Vector Machine, 支持向量机）作为验证使用主题分布作为文档表征时的分类性能。

多分类采用 One-vs-Rest 的方法，即将每个类别看作一个二分类问题，将其他类别的样本看作一个类别，这样就可以得到每个类别的分类器。在预测时，将待预测的样本输入到每个分类器中，得到每个分类器的预测结果，将预测结果中概率最大的类别作为最终的预测结果。

In [5]:
class Classification:
    def __init__(self, features: np.ndarray, target: List[int], test_size: float = 0.3):
        self.clf = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr')

        print('Splitting data...')
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(features, target, test_size=test_size, random_state=0)

        print('Training...')
        self.clf.fit(X_train, y_train)

        print('Predicting...')
        self.y_pred = self.clf.predict(X_test)
    
    def get_accuracy(self):
        return accuracy_score(self.y_test, self.y_pred)

### 3. 实验

在实验中，根据分词模式，分为以字为单位进行分词的实验，以及以词为单位进行分词的实验。

对于训练数据，让每个 token 等概率地作为起点，采样了 200 段长度在 500 到 1000 之间的段落。

在验证文档主题表征的分类性能时，使用 70% 的段落（也就是 140 个段落）训练 SVM 模型，剩下的 30 % 的段落作为测试集。（实验部分的具体代码见 `src\experiment.py`）

#### 3.1 以字为单位进行分词

以字为单位进行分词时，词表大小为 5299 。

分别在主题数为 10, 20, 30, 40, 50, 60 的情况下，训练 LDA 模型，得到主题分布，然后进行分类任务，最后的结果如下：

| 主题数 | 10   | 20   | 30   | 40   | 50   | 60   |
| ------ | ---- | ---- | ---- | ---- | ---- | ---- |
| 准确率 | 0.1833 | 0.1833 | 0.1833 | 0.2167 | 0.1833 | 0.1833 |

在主题数为 10 的 LDA 模型经过训练后，每个主题中概率最大的 10 个字为：

<table>
  <thead>
    <tr>
      <th  style="text-align: center;">主题</th>
      <th  style="text-align: center;" colspan="10">概率最大的10个字</th>
    </tr>
  </thead>
  <tbody>
      <tr><td>0</td><td>列</td><td>菊</td><td>掉</td><td>哭</td><td>宾</td><td>种</td><td>缺</td><td>给</td><td>拜</td><td>会</td></tr>
      <tr><td>1</td><td>辙</td><td>陛</td><td>于</td><td>舌</td><td>快</td><td>温</td><td>翻</td><td>张</td><td>纳</td><td>况</td></tr>
      <tr><td>2</td><td>奏</td><td>折</td><td>箫</td><td>臣</td><td>焕</td><td>洲</td><td>雨</td><td>兼</td><td>率</td><td>曲</td></tr>
      <tr><td>3</td><td>，</td><td>。</td><td>不</td><td>一</td><td>的</td><td>是</td><td>了</td><td>道</td><td>：</td><td>人</td></tr>
      <tr><td>4</td><td>税</td><td>浇</td><td>鸠</td><td>丰</td><td>卜</td><td>帆</td><td>殿</td><td>举</td><td>起</td><td>投</td></tr>
      <tr><td>5</td><td>珂</td><td>戚</td><td>胖</td><td>喝</td><td>将</td><td>囚</td><td>屋</td><td>彭</td><td>怨</td><td>迟</td></tr>
      <tr><td>6</td><td>修</td><td>刷</td><td>钟</td><td>贫</td><td>阿</td><td>珊</td><td>饷</td><td>哀</td><td>般</td><td>腹</td></tr>
      <tr><td>7</td><td>失</td><td>卷</td><td>词</td><td>求</td><td>梨</td><td>际</td><td>扎</td><td>莫</td><td>肌</td><td>铃</td></tr>
      <tr><td>8</td><td>徐</td><td>达</td><td>燕</td><td>邸</td><td>妃</td><td>蒸</td><td>鹅</td><td>旧</td><td>忠</td><td>读</td></tr>
      <tr><td>9</td><td>岱</td><td>右</td><td>切</td><td>飓</td><td>时</td><td>卑</td><td>泽</td><td>畅</td><td>玉</td><td>桑</td></tr>
  </tbody>
</table>


#### 3.2 以词为单位进行分词

以词为单位进行分词时，词表大小为 160090 。
由于在该词表大小下，训练一次 LDA 模型所需的时间很长（长达数小时），所以只在主题数为 10, 20, 30 的情况下训练 LDA 模型。最终分类任务的结果如下：

| 主题数 | 10   | 20   | 30   |
| ------ | ---- | ---- | ---- |
| 准确率 | 0.15 | 0.15 | 0.15 |

在主题数为 10 的 LDA 模型经过训练后，每个主题中概率最大的 10 个词为：

<table>
  <thead>
    <tr>
      <th  style="text-align: center;">主题</th>
      <th  style="text-align: center;" colspan="10">概率最大的10个词</th>
    </tr>
  </thead>
  <tbody>
      <tr><td>0</td><td>目光</td><td>阴险</td><td>到手</td><td>接续</td><td>弄坏了</td><td>鲍某</td><td>下山</td><td>敏君</td><td>虽存季</td><td>义 父</td></tr>
      <tr><td>1</td><td>畔</td><td>商店</td><td>辔头</td><td>前前后后</td><td>之于</td><td>点头</td><td>双目</td><td>举</td><td>换作</td><td>粗声粗 气</td></tr>
      <tr><td>2</td><td>一声令下</td><td>日夜</td><td>不弱</td><td>得失</td><td>深盼</td><td>来本寺</td><td>暗扣</td><td>十多年</td><td>南面</td><td>一车</td></tr>
      <tr><td>3</td><td>群中</td><td>要输</td><td>六仙</td><td>爱 惜</td><td>还活</td><td>王姑娘</td><td>拖延时间</td><td>声势</td><td>火线</td><td>头晕</td></tr>
      <tr><td>4</td><td>金批</td><td>欺软怕硬</td><td>不负</td><td>凝起</td><td>百思不得其解</td><td>欲求</td><td>四位</td><td>重围</td><td>剐</td><td>身份</td></tr>
      <tr><td>5</td><td>，</td><td>。</td><td>的</td><td>了</td><td>：</td><td>是</td><td>他</td><td>你</td><td>道</td><td>我</td></tr>
      <tr><td>6</td><td>违心之论</td><td>躲让</td><td>空等</td><td>後</td><td>进寺</td><td>总教</td><td>河边</td><td>恭谨</td><td>得色</td><td>寿比 南山</td></tr>
      <tr><td>7</td><td>恕</td><td>税</td><td>虽低</td><td>游玩</td><td>进门</td><td>收矿税</td><td>人品</td><td>门派</td><td>确然</td><td>撕成</td></tr>
      <tr><td>8</td><td>奔上</td><td>饼</td><td>中骂</td><td>声立断</td><td>元人</td><td>公平</td><td>视而不见</td><td>叫</td><td>话题</td><td>建宁</td></tr>
      <tr><td>9</td><td>上臂</td><td>一地</td><td>教士</td><td>性格</td><td>马超兴</td><td>上生</td><td>经行</td><td>响成</td><td>已然</td><td>如竹</td></tr>
  </tbody>
</table>

#### 3.3 实验结果中的一些问题

1. 对于从 LDA 模型得到的文档表征的分类性能，以字为单位分词的效果好与以词为单位分词的效果。

   这一现象可能有多方面因素导致，包括具体使用到的数据集不同以及词表大小不同等。对于词表，在两种分词方法下，大小存在数量级上的差异。并且分词情况下的词表相对于训练语料有点过大了。训练语料至多才 $200 \times 1000 = 2\times10^5$ 个词，但总共的词表大小却也有 $1.6\times10^5$ 个词。

2. 使用不同主题数的时候，模型的性能没有太大的改变。

   我认为这主要是数据量太小导致的。用于训练以及测试的样本数分别只有 140 和 60 个，但类别却有 16 个，所以样本数相对来说有点太小了。
   此外为了排除算法实现上的问题，我额外使用了 scikit-learn 中的 LDA 实现，但是结果也相差不多。
   
3. 主题中概率最大的几个词之间没有明显的联系。

   这一问题我认为也是由多方因素导致的，但首要的还是训练数据过少，使得无法抽取出明显的主题。

### 参考文献

[1] 李航.统计学习方法.北京:清华大学出版社,2012:390-401.